In [1]:
#Import dependencies
import pandas as pd
import numpy as np
from datetime import datetime
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:
#Constants
pd.set_option('display.max_columns', None)
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

college_park_coords = (38.9897, -76.9378)  # (lat, lon)
geolocator = Nominatim(user_agent="myGeocoder")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)


In [3]:
#Wrangling functions

def haversine(lat1, lon1, lat2, lon2):
    R = 3958.8  # Earth radius in miles
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

def get_lat_lon(address):
    location = geocode(address)
    if location:
        return pd.Series([location.latitude, location.longitude])
    else:
        return pd.Series([None, None])


# Function to convert 12-hour time to 24-hour format
def convert_to_military_time(time_str):
    try:
        return pd.to_datetime(time_str).strftime('%H:%M')
    except (ValueError, TypeError):
        return time_str
        
def create_row_name(day, frequency):
    if pd.isna(frequency) or frequency == 'every week':
        return str(day)
    else:
        if isinstance(frequency, str) and frequency.endswith(' of the Month'):
            beginning = frequency[:-len(' of the Month')].strip()
            end = ' of the month'
            return f"{beginning} {day}{end}"
        else:
            return str(day)


In [4]:
#Important dfs
images_list = pd.read_excel('../datasets/FoodPantryImages.xlsx')['image_address']
df = pd.read_csv('../datasets/CAFB_Markets_HOO.xlsx - CAFBAgencyHoursMOMKandCOMKResul.csv')

In [ ]:
#Get lat and long
address = df['Shipping Address'].str.replace(r'^[^\d]*', '', regex=True)
df[['latitude', 'longitude']] = address.apply(get_lat_lon)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('4-H Club Park and Planning 2411 Pinebrook Ave Hyattsville MD 20785',), **{}).
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/urllib3/connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/urllib3/connection.py", line 464, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/http/client.py", line 1428, in getresponse
    response.begin()
  File "/opt/anaconda3/lib/python3.12/http/client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/http/client.py", line 292, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
               ^^^^^^^^^^^^^^^^^^^^^

In [57]:
#Missing lat and long
df.loc[3, ['latitude', 'longitude']] = (38.90285, -77.01162)
df.loc[10, ['latitude', 'longitude']] = (38.87446, -77.01247)
df.loc[11, ['latitude', 'longitude']] = (38.94850, -77.03572)
df.loc[19, ['latitude', 'longitude']] = (38.39971, -77.44292)
df.loc[23, ['latitude', 'longitude']] = (38.85324, -77.13641)
df.loc[24, ['latitude', 'longitude']] = (38.64858, -77.35996)
df.loc[28, ['latitude', 'longitude']] = (38.83646, -77.20381)
df.loc[31, ['latitude', 'longitude']] = (38.80469, -77.16711)
df.loc[36, ['latitude', 'longitude']] = (38.85695, -77.13899)
df.loc[37, ['latitude', 'longitude']] = (38.90974, -77.26246)
df.loc[38, ['latitude', 'longitude']] = (38.75378, -77.08512)
df.loc[39, ['latitude', 'longitude']] = (38.79446, -77.52208)
df.loc[40, ['latitude', 'longitude']] = (38.92366, -76.89409)
df.loc[48, ['latitude', 'longitude']] = (38.89148, -76.90921)
df.loc[50, ['latitude', 'longitude']] = (38.90776, -76.91512)
df.loc[52, ['latitude', 'longitude']] = (38.90482, -76.91933)
df.loc[68, ['latitude', 'longitude']] = (38.83148, -76.93091)
df.loc[71, ['latitude', 'longitude']] = (39.03645, -77.04318)


df.loc[40, 'address'] = '2411 Pinebrook Ave Hyattsville MD 20785'
df.loc[48, 'address'] = '211 Maryland Park Drive Capital Heights MD 20743'

In [158]:
#Isolate columns
names = df['Agency Name'].str.replace(r'^[^ ]+ ', '', regex=True)
description = df['Food Format '].fillna('')
address = df['Shipping Address'].str.replace(r'^[^\d]*', '', regex=True)
contactPhone = ''
contactEmail = ''
imageUrl = np.random.choice(images_list, size=len(df))
walkingDistance = df.apply(
    lambda row: haversine(college_park_coords[0], college_park_coords[1], row['latitude'], row['longitude']),
    axis=1
)
offersDelivery = False
specialNotes = df['Choice Options '].fillna('')

hours = []
# Iterate over the DataFrame rows
for _, row in df.iterrows():
    day = row['Day of Week']
    start_time = convert_to_military_time(row['Starting Time'])
    end_time = convert_to_military_time(row['Ending Time'])
    frequency = row['Frequency']
    
    # Create the row name
    row_name = create_row_name(day, frequency)
    
    # Create a new DataFrame with 'open' and 'close' columns
    new_df = pd.DataFrame({
        'open': [start_time],
        'close': [end_time]
    }, index=[row_name])
    
    # Append the new DataFrame to the list
    hours.append(new_df)


In [159]:
#Recreate df
df1 = pd.DataFrame({
    'name': names.values,
    'description': description.values,
    'address': address.values,
    'latitude': df['latitude'],
    'longitude': df['longitude'],
    'contactPhone': contactPhone,
    'contactEmail': contactEmail,
    'imageUrl': imageUrl,
    'adminId': 1,
    'walkingDistance': walkingDistance.values,
    'offersDelivery': offersDelivery,
})

df1['openingHours'] = hours
df1['specialNotes'] = specialNotes
df1.at[46, 'openingHours'] = pd.concat([df1.loc[46, 'openingHours'], df1.loc[47, 'openingHours']], ignore_index=False)
df1.at[53, 'openingHours'] = pd.concat([df1.loc[53, 'openingHours'], df1.loc[54, 'openingHours']], ignore_index=False)
df1.drop([47, 54], inplace=True)

df1.head(40)

,name,description,address,latitude,longitude,contactPhone,contactEmail,imageUrl,adminId,walkingDistance,offersDelivery,openingHours,specialNotes
0,Academy of Hope,,2315 18th Pl NE Washington DC 20018,38.920754,-76.977739,,,https://streetviewpixels-pa.googleapis.com/v1/...,1,5.224783,False,open close 4th W...,
1,Ambassador Baptist Church,Pre-bagged or boxed groceries,1408 Minnesota Ave SE Washington DC 20020,38.866921,-76.984359,,,https://streetviewpixels-pa.googleapis.com/v1/...,1,8.844748,False,open clos...,Full Choice
2,Brotherhood of the Cross and Star,Loose groceries,6001 Georgia Ave NW Washington DC 20011,38.963116,-77.027420,,,https://streetviewpixels-pa.googleapis.com/v1/...,1,5.152401,False,open close 3rd Frid...,Full Choice
3,SeVerna,,43 K St NW Washington DC 20001,38.902850,-77.011620,,,https://lh3.googleusercontent.com/p/AF1QipNpqT...,1,7.193458,False,open close 3rd W...,
4,Lederer Gardens,Pre-bagged or boxed groceries,4801 Nannie Helen Burroughs Ave NE Washington ...,38.898503,-76.933551,,,https://lh3.googleusercontent.com/p/AF1QipO2DB...,1,6.305280,False,open close 2nd W...,Full Choice
5,Faith United Church of Christ,,4900 10th St NE Washington DC 20017,38.948552,-76.993036,,,https://lh3.googleusercontent.com/p/AF1QipNpqT...,1,4.109464,False,open close 3rd Th...,
6,Frederick Douglass Community Center,,2000 Alabama Ave SE Washington DC 20020,38.850814,-76.975832,,,https://streetviewpixels-pa.googleapis.com/v1/...,1,9.811588,False,open close 2nd Th...,
7,Congress Heights Family Success Center,Pre-bagged or boxed groceries,1345 Savannah St SE Washington DC 20032,38.843584,-76.986375,,,https://lh3.googleusercontent.com/p/AF1QipNpqT...,1,10.428016,False,open close 3rd Mond...,Full Choice
8,Garfield Terrace Resident Council,,2301 11th St NW Washington DC 20001,38.921518,-77.025815,,,https://streetviewpixels-pa.googleapis.com/v1/...,1,6.675132,False,open close 4th W...,
9,James Apartments Resident Council,,1425 N St NW Washington DC 20005,38.907476,-77.033105,,,https://lh3.googleusercontent.com/p/AF1QipO2DB...,1,7.648689,False,open close 1st Mond...,


In [41]:
df1.to_csv('temp.csv')

In [164]:
df2 = pd.read_csv('../datasets/CAFB_Shopping_Partners_HOO.xlsx - CAFBAgencyHoursofOperationwDeta.csv')
df2.head(5)

,External ID,Name,Status,Last SO Create Date,Agency Region,County/Ward,Shipping Address,Phone,Day of Week,Monthly Options,Starting Time,Ending Time,By Appointment Only,Food Pantry Requirements,Date of Last Verification,Distribution Models,Food Format,Additional Note on Hours of Operations
0,19079-PART-02,Children of Mine : Children of Mine,Active,1/21/2025,DC,DC Ward 8,"gloria ward-ravenell 2263 mount view place, se...",(202) 374-6029,Tuesday,2nd and 4th of the Month,8:00 AM,10:00 AM,No,NaN,10/2/2024,NaN,NaN,NaN
1,14265-PART-01,St. Martins Social Service : St. Martins Socia...,Active,1/3/2025,DC,DC Ward 5,Attn: St. Martin's Social Service 1908 North C...,(202) 232-1144,Wednesday,Every week,10:00 AM,11:30 AM,No,NaN,10/7/2024,NaN,NaN,walkups
2,18786-PART-01,Bethesda Cares : Bethesda Cares,Active,11/21/2024,MD,MD Montgomery County,Attn: Bethesda Cares 5033 Wilson Lane Bethesda...,(301) 907-9244,Monday,Every week,11:00 AM,1:00 PM,No,NaN,10/21/2024,NaN,NaN,walk-ins hotmeals
3,18786-PART-01,Bethesda Cares : Bethesda Cares,Active,11/21/2024,MD,MD Montgomery County,Attn: Bethesda Cares 5033 Wilson Lane Bethesda...,(301) 907-9244,Tuesday,Every week,11:00 AM,1:00 PM,No,NaN,10/21/2024,NaN,NaN,walk-ins hotmeals
4,18786-PART-01,Bethesda Cares : Bethesda Cares,Active,11/21/2024,MD,MD Montgomery County,Attn: Bethesda Cares 5033 Wilson Lane Bethesda...,(301) 907-9244,Wednesday,Every week,11:00 AM,1:00 PM,No,NaN,10/21/2024,NaN,NaN,walk in hot meals


In [166]:
# Group by both External ID and Distribution Models
grouped = df2.groupby(['External ID', 'Distribution Models'])

# Extract metadata: take the first row of each group
meta_df = grouped.first().reset_index()

# Build schedule DataFrames for each (External ID, Distribution Models) pair
store_schedules = {
    name: group.set_index('Day of Week')[['Monthly Options', 'Starting Time', 'Ending Time']]
    for name, group in grouped
}

# Add the schedule to meta_df
meta_df['schedule'] = meta_df.set_index(['External ID', 'Distribution Models']).index.map(store_schedules)

for idx, row1 in meta_df.iterrows():
    schedule_df = row1['schedule'].copy().reset_index()  # make 'Day of Week' a column

    
    schedule_df['Starting Time'] = schedule_df['Starting Time'].apply(convert_to_military_time)
    schedule_df['Ending Time'] = schedule_df['Ending Time'].apply(convert_to_military_time)

    schedule_df['Day of Week'] = schedule_df['Day of Week'].str.strip().str.capitalize()  # Clean the strings
    schedule_df['Day of Week'] = pd.Categorical(schedule_df['Day of Week'], categories=day_order, ordered=True)
    schedule_df = schedule_df.sort_values(['Day of Week', 'Starting Time'])
    
    new_index = [
        create_row_name(row['Day of Week'], row['Monthly Options'])
        for _, row in schedule_df.iterrows()
    ]
    schedule_df.index = new_index

    schedule_df.drop(columns=['Monthly Options', 'Day of Week'], inplace=True)
    schedule_df.columns = ['open', 'close']

    meta_df.at[idx, 'schedule'] = schedule_df
    
meta_df.head()


,External ID,Distribution Models,Name,Status,Last SO Create Date,Agency Region,County/Ward,Shipping Address,Phone,Day of Week,Monthly Options,Starting Time,Ending Time,By Appointment Only,Food Pantry Requirements,Date of Last Verification,Food Format,Additional Note on Hours of Operations,schedule
0,14030-PART-01,Home Delivery,SOME : So Others Might Eat,Active,1/3/2025,DC,DC Ward 5,Attn: SOME 71 O ST NW Washington DC 20002,(202) 235-1472,Monday,Every week,12:00 PM,2:00 PM,No,"ID,Income,Zip Code",10/2/2024,Prepared meals,Not open to public.,open close Monday 12:00 14:00
1,14030-PART-01,Walk up,SOME : So Others Might Eat,Active,1/3/2025,DC,DC Ward 5,Attn: SOME 71 O ST NW Washington DC 20002,(202) 235-1472,Monday,Every week,9:00 AM,11:00 AM,No,"ID,Income,Zip Code",10/2/2024,Loose groceries,None,open close Monday 09:00 11:0...
2,14040-PART-01,Walk up,Community Family Life Services : Community Fam...,Active,1/30/2025,DC,DC Ward 6,Attn: Community Family Life Services 305 E Str...,(202) 347-0511,Tuesday,Every week,10:00 AM,12:00 PM,No,ID,9/25/2024,Pre-bagged or boxed groceries,None,open close Tuesday 10:00 12:00
3,14050-PART-01,Walk up,Bread for the City : Bread for the City SE Center,Active,2/24/2025,DC,DC Ward 8,Attn: Bread For The City SE 1700 Good Hope Roa...,(202) 773-2308,Monday,Every week,9:00 AM,3:00 PM,No,"ID,Zip Code",9/20/2024,Loose groceries,DC Residents Only,open close Monday 09:00 15:0...
4,14050-PART-02,Walk up,Bread for the City : Bread For The City NW Center,Active,2/24/2025,DC,DC Ward 2,Attn: Bread For The City NW 1525 7th St. NW Wa...,(202) 773-2308,Monday,Every week,9:00 AM,3:00 PM,No,"ID,Zip Code",9/19/2024,Loose groceries,DC Residents Only,open close Monday 09:00 15:0...


In [169]:
#Get adress before getting coordinates
address = meta_df['Shipping Address'].str.replace(r'^[^\d]*', '', regex=True)


In [168]:
#Don't run
# 
# 
# Get coordinates
meta_df[['latitude', 'longitude']] = address.apply(get_lat_lon)

KeyboardInterrupt: 

In [170]:
#No coordinates
meta_df.loc[3, ['latitude', 'longitude']] = (38.86634, 76.98038)
meta_df.loc[5, ['latitude', 'longitude']] = (38.89990, 77.04720)
meta_df.loc[6, ['latitude', 'longitude']] = (38.89065, 76.93169)
meta_df.loc[7, ['latitude', 'longitude']] = (38.90686, 76.83885)
meta_df.loc[14, ['latitude', 'longitude']] = (38.90187, 77.00953)
meta_df.loc[26, ['latitude', 'longitude']] = (38.94684, 77.03691)
meta_df.loc[27, ['latitude', 'longitude']] = (38.84611, 76.86846)
meta_df.loc[30, ['latitude', 'longitude']] = (38.96862, 77.34277)
meta_df.loc[34, ['latitude', 'longitude']] = (38.96612, 76.98619)
meta_df.loc[35, ['latitude', 'longitude']] = (38.95594, 77.03593)
meta_df.loc[36, ['latitude', 'longitude']] = (38.85761, 77.13846)
meta_df.loc[37, ['latitude', 'longitude']] = (38.86299, 76.99123)
meta_df.loc[38, ['latitude', 'longitude']] = (38.86299, 76.99123)
meta_df.loc[41, ['latitude', 'longitude']] = (38.66643, 77.33503)
meta_df.loc[44, ['latitude', 'longitude']] = (38.75378, 77.08512)
meta_df.loc[45, ['latitude', 'longitude']] = (38.75378, 77.08512)
meta_df.loc[46, ['latitude', 'longitude']] = (38.77445, 77.01391)
meta_df.loc[48, ['latitude', 'longitude']] = (38.89012, 76.98104)
meta_df.loc[50, ['latitude', 'longitude']] = (38.86962, 76.95956)
meta_df.loc[53, ['latitude', 'longitude']] = (39.00234, 77.21416)
meta_df.loc[54, ['latitude', 'longitude']] = (38.88383, 77.44306)
meta_df.loc[63, ['latitude', 'longitude']] = (38.88430, 76.92458)
meta_df.loc[65, ['latitude', 'longitude']] = (38.83148, 76.93091)
meta_df.loc[66, ['latitude', 'longitude']] = (38.89148, 76.90921)
meta_df.loc[67, ['latitude', 'longitude']] = (38.74377, 76.08161)
meta_df.loc[72, ['latitude', 'longitude']] = (38.91809, 76.84369)
meta_df.loc[76, ['latitude', 'longitude']] = (38.88354, 76.89629)
meta_df.loc[79, ['latitude', 'longitude']] = (38.99083, 77.02755)
meta_df.loc[80, ['latitude', 'longitude']] = (39.05471, 76.96907)
meta_df.loc[81, ['latitude', 'longitude']] = (38.66047, 76.77234)
meta_df.loc[87, ['latitude', 'longitude']] = (38.86534, 76.92365)
meta_df.loc[89, ['latitude', 'longitude']] = (38.98888, 76.82493)
meta_df.loc[92, ['latitude', 'longitude']] = (38.94682, 76.80098)
meta_df.loc[93, ['latitude', 'longitude']] = (38.90215, 76.90578)
meta_df.loc[94, ['latitude', 'longitude']] = (38.74692, 77.08193)
meta_df.loc[95, ['latitude', 'longitude']] = (38.65208, 76.25979)
meta_df.loc[96, ['latitude', 'longitude']] = (38.94282, 77.41688)
meta_df.loc[101, ['latitude', 'longitude']] = (38.85695, 77.13899)
meta_df.loc[105, ['latitude', 'longitude']] = (38.96470, 76.86554)
meta_df.loc[119, ['latitude', 'longitude']] = (38.80981, 77.26279)
meta_df.loc[122, ['latitude', 'longitude']] = (38.88461, 76.73073)
meta_df.loc[123, ['latitude', 'longitude']] = (38.88461, 76.73073)
meta_df.loc[125, ['latitude', 'longitude']] = (38.83346, 76.86675)
meta_df.loc[126, ['latitude', 'longitude']] = (38.97216, 76.38557)
meta_df.loc[133, ['latitude', 'longitude']] = (39.17402, 77.20577)
meta_df.loc[136, ['latitude', 'longitude']] = (38.79054, 77.59805)
meta_df.loc[137, ['latitude', 'longitude']] = (38.79054, 77.59805)
meta_df.loc[138, ['latitude', 'longitude']] = (38.79054, 77.59805)
meta_df.loc[141, ['latitude', 'longitude']] = (38.90578, 76.99492)
meta_df.loc[143, ['latitude', 'longitude']] = (38.82548, 76.74886)
meta_df.loc[145, ['latitude', 'longitude']] = (38.86171, 77.14652)
meta_df.loc[147, ['latitude', 'longitude']] = (38.67215, 77.26169)
meta_df.loc[149, ['latitude', 'longitude']] = (38.83154, 77.20901)
meta_df.loc[151, ['latitude', 'longitude']] = (39.08854, 76.85273)
meta_df.loc[152, ['latitude', 'longitude']] = (39.00188, 76.87637)
meta_df.loc[165, ['latitude', 'longitude']] = (38.96694, 77.35265)
meta_df.loc[167, ['latitude', 'longitude']] = (39.11537, 76.62088)
meta_df.loc[168, ['latitude', 'longitude']] = (39.17217, 77.16221)
meta_df.loc[170, ['latitude', 'longitude']] = (38.96524, 76.84091)
meta_df.loc[172, ['latitude', 'longitude']] = (38.86160, 77.09346)
meta_df.loc[175, ['latitude', 'longitude']] = (38.76133, 77.44914)
meta_df.loc[178, ['latitude', 'longitude']] = (39.00871, 76.99750)
meta_df.loc[183, ['latitude', 'longitude']] = (38.72297, 77.11779)
meta_df.loc[185, ['latitude', 'longitude']] = (38.76553, 76.82876)
meta_df.loc[186, ['latitude', 'longitude']] = (38.76553, 76.82876)
meta_df.loc[200, ['latitude', 'longitude']] = (38.85784, 77.14192)
meta_df.loc[209, ['latitude', 'longitude']] = (38.75172, 77.08221)
meta_df.loc[210, ['latitude', 'longitude']] = (38.75172, 77.08221)
meta_df.loc[211, ['latitude', 'longitude']] = (38.84978, 76.88064)
meta_df.loc[216, ['latitude', 'longitude']] = (38.77873, 76.88857)
meta_df.loc[218, ['latitude', 'longitude']] = (38.88091, 76.85656)
meta_df.loc[226, ['latitude', 'longitude']] = (38.90194, 77.04453)
meta_df.loc[228, ['latitude', 'longitude']] = (38.90630, 76.84380)
meta_df.loc[230, ['latitude', 'longitude']] = (38.77250, 76.88190)
meta_df.loc[234, ['latitude', 'longitude']] = (39.11100, 77.06529)
meta_df.loc[235, ['latitude', 'longitude']] = (38.88843, 76.88110)
meta_df.loc[242, ['latitude', 'longitude']] = (38.89065, 76.93169)
meta_df.loc[258, ['latitude', 'longitude']] = (39.17895, 77.26847)
meta_df.loc[259, ['latitude', 'longitude']] = (39.17895, 77.26847)
meta_df.loc[261, ['latitude', 'longitude']] = (39.09478, 77.04554)
meta_df.loc[262, ['latitude', 'longitude']] = (39.09478, 77.04554)
meta_df.loc[265, ['latitude', 'longitude']] = (38.87885, 77.01680)
meta_df.loc[271, ['latitude', 'longitude']] = (38.73304, 77.06855)
meta_df.loc[272, ['latitude', 'longitude']] = (38.92068, 77.03165)
meta_df.loc[281, ['latitude', 'longitude']] = (38.91764, 77.3000)
meta_df.loc[283, ['latitude', 'longitude']] = (38.92698, 77.03282)
meta_df.loc[284, ['latitude', 'longitude']] = (38.92698, 77.03282)
meta_df.loc[285, ['latitude', 'longitude']] = (38.92698, 77.03282)
meta_df.loc[286, ['latitude', 'longitude']] = (38.86820, 77.22926)
meta_df.loc[290, ['latitude', 'longitude']] = (38.96828, 77.33910)
meta_df.loc[291, ['latitude', 'longitude']] = (38.71585, 77.19248)
meta_df.loc[294, ['latitude', 'longitude']] = (38.83534, 77.45543)
meta_df.loc[295, ['latitude', 'longitude']] = (38.83534, 77.45543)
meta_df.loc[297, ['latitude', 'longitude']] = (39.12102, 77.06901)
meta_df.loc[298, ['latitude', 'longitude']] = (39.12102, 77.06901)
meta_df.loc[300, ['latitude', 'longitude']] = (39.10043, 76.81451)
meta_df.loc[301, ['latitude', 'longitude']] = (38.91387, 76.79820)
meta_df.loc[302, ['latitude', 'longitude']] = (38.88637, 76.82032)
meta_df.loc[306, ['latitude', 'longitude']] = (38.87122, 77.19074)
meta_df.loc[312, ['latitude', 'longitude']] = (38.64098, 77.26599)
meta_df.loc[317, ['latitude', 'longitude']] = (39.09291, 77.14419)
meta_df.loc[318, ['latitude', 'longitude']] = (39.09291, 77.14419)
meta_df.loc[325, ['latitude', 'longitude']] = (38.64483, 77.30078)
meta_df.loc[327, ['latitude', 'longitude']] = (38.64643, 77.32125)
meta_df.loc[328, ['latitude', 'longitude']] = (38.67514, 77.39712)
meta_df.loc[330, ['latitude', 'longitude']] = (38.85707, 76.98951)


In [171]:
names = meta_df['Name'].str.split(':').str[1].str.strip()
description = meta_df['Food Format '].fillna('')
contactPhone = meta_df['Phone'].fillna('')
contactEmail = ''
imageUrl = np.random.choice(images_list, size=len(meta_df))
walkingDistance = meta_df.apply(
    lambda row: haversine(college_park_coords[0], college_park_coords[1], row['latitude'], row['longitude']),
    axis=1
)
offersDelivery = meta_df['Distribution Models'].str.contains('Home Delivery', na=False)
specialNotes = meta_df['Additional Note on Hours of Operations'].fillna('')



In [172]:
df2 = pd.DataFrame({
    'name': names.values,
    'description': description.values,
    'address': address.values,
    'latitude': meta_df['latitude'],
    'longitude': meta_df['longitude'],
    'contactPhone': contactPhone.values,
    'contactEmail': contactEmail,
    'imageUrl': imageUrl,
    'adminId': 1,
    'walkingDistance': walkingDistance.values,
    'offersDelivery': offersDelivery.values,
    'openingHours': meta_df['schedule'],
    'specialNotes': specialNotes.values
})



In [173]:
df2.loc[3, 'address'] = '1700 Marion Barry Ave SE Washington DC 20020'
df2.loc[5, 'address'] = '800 21st Street NW Washington DC 20052'
df2.loc[6, 'address'] = '4929 Ames St NE Washington DC 20019'
df2.loc[46, 'address'] = '8310 Ft Foote Road Fort Washington MD 20744'
df2.loc[48, 'address'] = '1636 East Capitol Street SE Washington DC 20003'
df2.loc[66, 'address'] = '211 Maryland Park Dr Capitol Heights MD 20743'
df2.loc[95, 'address'] = '13950 Richmond Highway Woodbridge VA 22191'
df2.loc[151, 'address'] = '8740 Cherry Lane Laurel MD 20707'
df2.loc[209, 'address'] = '2912 Woodlawn Trail Alexandria VA 22306'
df2.loc[210, 'address'] = '2912 Woodlawn Trail Alexandria VA 22306'
df2.loc[306, 'address'] = '2929 Graham Road Falls Church VA 22042'
df2.loc[312, 'address'] = '14339 Richmond Highway Woodbridge VA 22191'

In [174]:
df2.loc[200:250, ['address', 'latitude', 'longitude']]

,address,latitude,longitude
200,6066 Leesburg Pike Falls Church VA 22041,38.85784,77.14192
201,12805 Georgia Avenue Silver Spring MD 20906,NaN,NaN
202,12805 Georgia Avenue Silver Spring MD 20906,NaN,NaN
203,12805 Georgia Avenue Silver Spring MD 20906,NaN,NaN
204,12805 Georgia Avenue Silver Spring MD 20906,NaN,NaN
205,7513 Northern Avenue Glenn Dale MD 20769,NaN,NaN
206,7513 Northern Avenue Glenn Dale MD 20769,NaN,NaN
207,5150 Fillmore Ave Alexandria VA 22311,NaN,NaN
208,13224 Franklin Farm Rd Herndon VA 20171,NaN,NaN
209,2912 Woodlawn Trail Alexandria VA 22306,38.75172,77.08221


In [175]:
df2

,name,description,address,latitude,longitude,contactPhone,contactEmail,imageUrl,adminId,walkingDistance,offersDelivery,openingHours,specialNotes
0,So Others Might Eat,Prepared meals,71 O ST NW Washington DC 20002,NaN,NaN,(202) 235-1472,,https://lh3.googleusercontent.com/p/AF1QipO2DB...,1,NaN,True,open close Monday 12:00 14:00,Not open to public.
1,So Others Might Eat,Loose groceries,71 O ST NW Washington DC 20002,NaN,NaN,(202) 235-1472,,https://streetviewpixels-pa.googleapis.com/v1/...,1,NaN,False,open close Monday 09:00 11:0...,
2,Community Family Life Services,Pre-bagged or boxed groceries,305 E Street NW Washington DC 20001,NaN,NaN,(202) 347-0511,,https://lh3.googleusercontent.com/p/AF1QipOgEU...,1,NaN,False,open close Tuesday 10:00 12:00,
3,Bread for the City SE Center,Loose groceries,1700 Marion Barry Ave SE Washington DC 20020,38.86634,76.98038,(202) 773-2308,,https://streetviewpixels-pa.googleapis.com/v1/...,1,6809.498126,False,open close Monday 09:00 15:0...,DC Residents Only
4,Bread For The City NW Center,Loose groceries,1525 7th St. NW Washington DC 20001,NaN,NaN,(202) 773-2308,,https://lh3.googleusercontent.com/p/AF1QipNpqT...,1,NaN,False,open close Monday 09:00 15:0...,DC Residents Only
...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,Holy Family Catholic Church Food Pantry,Pre-bagged or boxed groceries,14160 Ferndale Road Dale City VA 22193,38.64643,77.32125,(703) 459-7156,,https://lh3.googleusercontent.com/p/AF1QipO2DB...,1,6830.078165,False,open close Monday 16:00 17:3...,
328,Sacred Heart Catholic Church,Pre-bagged or boxed groceries,12975 Purcell Road Manassas VA 20112,38.67514,77.39712,(703) 590-0030,,https://streetviewpixels-pa.googleapis.com/v1/...,1,6829.609998,False,open close Saturday 08:00 09:30,
329,Manassas Baptist Church,Pre-bagged or boxed groceries,8730 Sudley Road Manassas VA 20110,NaN,NaN,(703) 361-2146,,https://lh3.googleusercontent.com/p/AF1QipOgEU...,1,NaN,False,open clo...,
330,Washington View Apartments,Loose groceries,2629 Douglass Rd SE Washington DC DC 20020,38.85707,76.98951,(202) 621-1600,,https://lh3.googleusercontent.com/p/AF1QipNpqT...,1,6810.268900,False,open clos...,


In [145]:
df3 = pd.concat([df1,df2], axis=0).reset_index().drop(columns='index')

In [157]:
def opening_hours_df_to_dict(df):
    return {
        day.lower(): {'open': row['open'], 'close': row['close']}
        for day, row in df.iterrows()
    }

df3['openingHours'] = df3['openingHours'].apply(opening_hours_df_to_dict)


KeyError: 'open'

In [133]:
df3.to_json('output.json', orient='records', indent=2)

json_str = df3.to_json(orient='records', indent=2)
json_str = json_str.replace('\\/', '/')

# Save to file
with open('output.json', 'w') as f:
    f.write(json_str)